# This notebook accompanies the lecture slides for GEL160 Lecture 21

In [ ]:
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import cv2 as cv

# Part 1: smoothing and filtering

This section demonstrates how we can threshold and then filter the Briones image from last week's lab.

In [ ]:
# Load the briones image
briones = io.imread('../lab7/briones.tiff')
# Throw away the alpha channel
briones = briones[:,:,:-1]
# Convert to greyscale
from skimage import color
briones_bw = color.rgb2gray(briones)
# show rgb and grayscale side by side
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].imshow(briones)
ax[0].set_xticks([]) # Note - this is one way  to remove the ticks and tick labels for a cleaner appearance
ax[0].set_yticks([])
ax[1].imshow(briones_bw,cmap='gray')
ax[1].set_xticks([])
ax[1].set_yticks([])
plt.tight_layout()
plt.show()
# show the histogram of the greyscale image
plt.figure()
plt.hist(briones_bw.ravel(),256)
plt.show()
# specify threshold value
threshold = 4.0/255
briones_mask = briones_bw <= threshold
plt.figure()
plt.imshow(briones_mask,cmap='gray')
plt.show()

## Median filter
The median filter can be applied to grayscale images as well as binary images. This filter looks at each pixel in the image and assigns to each pixel the median value from all of the neighboring pixels

**This filter is good for removing outlier pixel values as the median tends to discard extreme values. It has a tendency to retain sharp contrasts that existed in the original image.**

In the code below, I define the 'neighborhood size' as 11. We create a structuring element that is a 11x11 matrix containing ones. This has the effect of taking the median over all of the pixels within an 11x11 box centered on each pixel. Try increasing and decreasing the size of the neighborhood to develop a feel for how this affects the filtered image.

In [ ]:
from skimage import filters
neighborhood_size = 11
structuring_element = np.ones((neighborhood_size,neighborhood_size))

briones_medfilt = filters.median(briones_bw,structuring_element)
fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_bw,cmap='gray')
ax[1].imshow(briones_medfilt,cmap='gray')
plt.show()

In [ ]:
# Experiment with thresholding after median filter:
threshold = 2.0/255
briones_medfilt_mask = briones_medfilt <= threshold

fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_mask,cmap='gray')
ax[1].imshow(briones_medfilt_mask,cmap='gray')
plt.show()

## Gaussian filter

The Gaussian filter applies an averaging scheme to the color associated with each pixel. Each pixel is assigned a weighted average of all of the other pixels in the image, where the weight is calculated based on a Gaussian function with a specifed shape parameter in numbers of pixels.

**The Gaussian filter can be applied to both color and greyscale images**
**This filter is useful for blurring an image. It will blur boundaries and produce a less-sharp image than the starting image**

In [ ]:
briones_gauss = filters.gaussian(briones,sigma=2)
fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones)
ax[1].imshow(briones_gauss)
plt.show()

# Part 2: Morphological operations

In this part we will look at the morphological operations 'erosion', 'dilation', 'opening' and 'closing'.
I will continue to use the *thresholded* briones example here

In [ ]:
kernel = np.ones((5,5),np.uint8)  # note this is a 5x5 ‘square’ kernel
# Note: multiply by 1.0 to convert from True/False to float
briones_eroded = cv.erode(briones_mask*1.0,kernel,iterations = 1) 

fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_mask,cmap='gray')
ax[1].imshow(briones_eroded,cmap='gray')
plt.show()

## Dilation

In [ ]:
kernel = np.ones((5,5),np.uint8)  # note this is a 5x5 ‘square’ kernel
# Note: multiply by 1.0 to convert from True/False to float
briones_dilated = cv.dilate(briones_mask*1.0,kernel,iterations = 1) 

fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_mask,cmap='gray')
ax[1].imshow(briones_dilated,cmap='gray')
plt.show()

# Opening

In [ ]:
kernel = np.ones((5,5),np.uint8)  # note this is a 5x5 ‘square’ kernel

briones_open = cv.morphologyEx(briones_mask*1.0, cv.MORPH_OPEN, kernel)
fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_mask,cmap='gray')
ax[1].imshow(briones_open,cmap='gray')
plt.show()

## Morphological Closing

In [ ]:
kernel = np.ones((5,5),np.uint8)  # note this is a 5x5 ‘square’ kernel

briones_close = cv.morphologyEx(briones_mask*1.0, cv.MORPH_CLOSE, kernel)
fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].imshow(briones_mask,cmap='gray')
ax[1].imshow(briones_close,cmap='gray')
plt.show()

## Morphological operations: Choosing the  structuring element

In [ ]:
# specify the size of the element in pixels
s = 5;
rect = cv.getStructuringElement(cv.MORPH_RECT,(s,s))
circ = cv.getStructuringElement(cv.MORPH_ELLIPSE,(s,s))
cross = cv.getStructuringElement(cv.MORPH_CROSS,(s,s))

fig, ax = plt.subplots(3,1,figsize=(10,3))
ax[0].imshow(rect,cmap='gray',vmin=0.0,vmax=1.0)
ax[1].imshow(circ,cmap='gray',vmin=0.0,vmax=1.0)
ax[2].imshow(cross,cmap='gray',vmin=0.0,vmax=1.0)
for i in ax:
    i.set_xticks([])
    i.set_yticks([])
plt.show()

# demonstrate Opening with each kernel:
briones_rect = cv.morphologyEx(briones_mask*1.0, cv.MORPH_DILATE, rect)
briones_circ = cv.morphologyEx(briones_mask*1.0, cv.MORPH_DILATE, circ)
briones_cross = cv.morphologyEx(briones_mask*1.0, cv.MORPH_DILATE, cross)
fig,ax = plt.subplots(3,1,figsize=(6,12))
ax[0].imshow(briones_rect,cmap='gray')
ax[1].imshow(briones_circ,cmap='gray')
ax[2].imshow(briones_cross,cmap='gray')
for i in ax:
    i.set_xticks([])
    i.set_yticks([])
plt.show()

# Part 3: Analyzing connected components

Let's load an image, threshold it to produce a binary mask, and then analyze the properties of the 'connected' components of the image.

In [ ]:
ice = io.imread('https://svs.gsfc.nasa.gov/vis/a010000/a011500/a011539/fc-1280.jpg')
ice_bw = color.rgb2gray(ice)
fig,ax = plt.subplots(1,3,figsize=(12,4))
ax[0].imshow(ice)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title('Ice - original')
ax[1].imshow(ice_bw,cmap='gray')
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title('Ice - grayscale')
ax[2].hist(ice_bw.flatten(),255)
ax[2].set_title('Histogram')
plt.tight_layout()
plt.show()

In [ ]:
# define threshold value for ice
%matplotlib notebook
tval = 0.6
ice_mask = 1.0*(ice_bw > tval)
fig,ax = plt.subplots(1,2)
ax[0].imshow(ice_bw,cmap='gray')
ax[1].imshow(ice_mask,cmap='gray')
plt.show()

In [ ]:
# Use morphological erosion to remove the very small, isolated blocks and focus on the big icebergs
%matplotlib inline
s=3
circ = cv.getStructuringElement(cv.MORPH_ELLIPSE,(s,s))
ice_open = cv.morphologyEx(ice_mask*1.0, cv.MORPH_OPEN, kernel)

fig,ax = plt.subplots(1,2,figsize=(12,4))
ax[0].imshow(ice_mask,cmap='gray')
ax[0].set_title('Thresholded image')
ax[1].imshow(ice_open,cmap='gray')
ax[1].set_title('After morphological opening')
for i in ax:
    i.set_xticks([])
    i.set_yticks([])
plt.show()

In [ ]:
# Label the connected regions
from skimage import measure
labels = measure.label(ice_open) # This does the labeling!

fig,ax = plt.subplots(1,2,figsize=(12,4))
ax[0].imshow(ice_mask,cmap='gray')
ax[0].set_title('After morphological opening')
ax[1].imshow(labels,cmap='jet')
ax[1].set_title('Connected components')

for i in ax:
    i.set_xticks([])
    i.set_yticks([])
plt.show()

# Retrieve properties of regions
The properties of the connected components that we can readily measure are described here:

https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops

In [ ]:
from skimage.measure import regionprops
properties = regionprops(labels,intensity_image=labels)
# Let's look at the area of each component
areas = []
for region in properties:
    areas.append(region.area)
areas = np.array(areas)
print(areas)

# Sort based on area
sort_order = np.argsort(areas)
print("Sorted order of regions:",sort_order)
i = sort_order[-1] # -1 will give us the largest region, -2 the second-largest etc...
# Get additional property (centroid location)
centroid = properties[i].weighted_centroid
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].imshow(labels)
ax[0].plot(centroid[1],centroid[0],'rx') # NOTE centroid is defined by (row,col) so we need to plot it as (x,y)=(col,row)
ax[0].set_title('Region {:d} area {:d} pixels'.format(i,areas[i]))
ax[1].imshow(properties[i].filled_image,cmap='gray')
ax[1].set_title('close-up of region')
plt.show()

In [ ]:
# Make a histogram of thee iceberg sizes. Let's remove the largest value first, which corresponds to the glacier
sorted_areas = np.sort(areas)
sorted_areas = sorted_areas[:-1]
plt.figure()
plt.hist(sorted_areas,20)
plt.xlabel('Area (pixels)')
plt.ylabel('Number')
plt.show()

# Part 4: Image Quantization using k-means clustering

In [ ]:
# Conceptually demonstrate k-means clustering in 2D
# Generate some random vectors stored in x and y:
x = np.concatenate([2.0+1.0*np.random.rand(10),3.0+2.0*np.random.rand(15)])
y = np.concatenate([4.0+2.5*np.random.rand(10),1.0+2.5*np.random.rand(15)])

plt.figure
plt.plot(x,y,'.')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# Apply k-means clustering to the 2D data here generated above. This example is based closely on the OpenCV documentation:
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html (section 2)
import cv2
# This will combine x and y into a single matrix. Each column of the matrix will contain a single (x,y) pair.
# The first row in the matrix will contain all of the x-values
# The seecond row in the matrix will contain all of the y-values
observations = np.vstack((x,y)).transpose()
# NOTE: OpenCV will only allow observation vectors stored as float32. 
# You can convert from one datatype to another. Here we are converting observations to 'float32':
observations = np.array(observations,dtype='float32')

# Set the 'criteria' for the k-means search:
# The criteria are a list of three quantities (termination criteria, maximum iterations, tolerance)
# The algorithm is iterative and it is a good idea to experiment with increasing the maximum iterations and decreasing the tolerance
# Make sure that the results are not changing substantially as you increase these quantities
# There is no guarantee that you will recover the optimal solution, but the algorithm generally performs very well.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)
# specify the number of means:
k=2
ret,label,center=cv2.kmeans(observations,k,None,criteria,1000,cv2.KMEANS_RANDOM_CENTERS)
print('Distance=',ret)
print('labels=',label)
print('center=',center)

In [ ]:
# Visualize the results of the k-means procedure:
plt.figure()
for i in range(k):
    mask = label==i
    mask = mask.flatten()
    h=plt.plot(observations[mask,0],observations[mask,1],'.',label='data in cluster {:d}'.format(i))
    # Plot the center of this cluster using a square
    plt.plot(center[i,0],center[i,1],'s',c=h[0].get_color(),label='Center {:d}'.format(i))
plt.legend()
plt.show()

In [ ]:
# Application to an image - in this case a satellite photo of mono lake:
rgb_values = briones.reshape(-1,3)
rgb_values = np.float32(rgb_values)
# specify number of clusters
k=4
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100000, 1.0)
ret,label,center=cv2.kmeans(rgb_values,k,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
print('Distance=',ret)
print('center=',center)
# Now convert back into uint8, and make original image
center = np.uint8(center)        # Convert center values back to 8 bits per channel
restored_pixels = center[label.flatten()]    
briones_kmeans = restored_pixels.reshape((briones.shape))
# show the results:
fig,ax = plt.subplots(1,3,figsize=(12,6))
ax[0].imshow(briones)
ax[0].set_title('Original image')
ax[1].imshow(briones_kmeans)
ax[1].set_title('Quantized image')
h=ax[2].imshow(label.reshape(briones.shape[0:2]),cmap='jet')

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax[2])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
ax[2].set_title('Cluster number (k={:d})'.format(k))
for i in ax:
    i.set_xticks([])
    i.set_yticks([])
plt.show()

In [ ]:
label.reshape(briones.shape[0:2])
label.reshape(briones.shape[0:2])